# Wrangle and Analyze Data Project - WeRateDogs case

In [1]:
import requests
import pandas as pd
import numpy as np
import tweepy
import time
import json
from timeit import default_timer as timer
import re
from functools import reduce
import plotly.express as px

## Data Wrangling section

### Gathering

In [2]:
# reading the WeRateDogs twitter archive
df_dogs = pd.read_csv('twitter-archive-enhanced.csv')

In [3]:
# Dowloading programmatically using Requests library
r = requests.get('https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv')
open('image-predictions.tsv', 'wb').write(r.content)

335079

In [4]:
# reading the file downloaded in the previous step
pictures_df= pd.read_csv('image-predictions.tsv',sep = '\t')

In [5]:
# storing the tweet_id from the WeRateDogs tweitter archive to use it in the future to get the likes ,count and other variables
tweet_ids = df_dogs['tweet_id']

In [6]:
# This piece of code is open seeing that the twitter account was created for the gathering process

consumer_key = 'ibWILk63sUtGvbg1Rp4zMieH2'
consumer_secret = 'jMFv72oSBNixKvkXRriDR3QvmqqSWCLHQTO5pKdQlezv4rLETv'
access_token = '1364748608063569929-epQWQBKdh8pU3KszioEiGc6FxZ7OI3'
access_secret = 'GCAuQom6GjfcwOZNJAYFEdUsqWwGeRNXb7Slj9z2gQPTG'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth , wait_on_rate_limit = True , wait_on_rate_limit_notify = True )

count = 0
start = timer()

with open('tweet_json.txt','w') as outfile:
    for tweet_id in tweet_ids:
        count+=1
        print(str(count)+ ": " + str(tweet_id)) 
        try:
            tweet = api.get_status(tweet_id, tweet_mode = 'extended')
            print('Success')
            json.dump(tweet._json,outfile)
            outfile.write('\n')
        except:
            pass
end = timer()
print(end-start)


In [7]:
#converting the json file into a data frame that comes from the previous code lines
tuits = []
for line in open('tweet_json.txt' , 'r'):
    tuits.append(json.loads(line))
df2 = pd.DataFrame(tuits)

In [8]:
# storing just the minimum required columns in a new df
tweets_df = df2[['id', 'retweet_count', 'favorite_count']]

### Assessing

#### detected quality in tidiness issues, we are not going to clean all of this

##### quality 
* (1) column name in df_dogs : there are names that are not names, all of them start with lower case so we will replace them with 'None'
* (2) column name in df_dogs : there are "None" values
* (2) column rating_numerator: there are "None" values
* (2) column rating_denominator: there are "None" values
* (2) column dogg, floofer, pupper & puppo: there are "None" values
* (3) completeness : we have in the df_dogs 2356 rows, while in pictures_df we have just 2075 and in tweets_df we have 2331
* (4) column name in df_dogs : should be string type
* (4) timestamp in df_dogs : should be converted to time type
* (5) retweets : we should drop the retweets and replays
* (6) pictures_df : we have rows where all the pictures do not contain dogs, so we will drop those rows
* (7) column rating_numerator & rating_denominator: there are values that donnot correspond to what the text says
* (8) breed in pictures_df : we will choose the most accurate breed according to the algorithm iterations

#### tidiness
* (1) column doggo , floofer, pupper & puppo : they could be stored in one column (categorical one)
* (2) timestamp : timezone and date is in the same column, we should split them
* (3) mergin in one DF the 3 DFs

In [9]:
df_dogs

tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
0     892420643555336193                    NaN                  NaN   
1     892177421306343426                    NaN                  NaN   
2     891815181378084864                    NaN                  NaN   
3     891689557279858688                    NaN                  NaN   
4     891327558926688256                    NaN                  NaN   
...                  ...                    ...                  ...   
2351  666049248165822465                    NaN                  NaN   
2352  666044226329800704                    NaN                  NaN   
2353  666033412701032449                    NaN                  NaN   
2354  666029285002620928                    NaN                  NaN   
2355  666020888022790149                    NaN                  NaN   

                      timestamp  \
0     2017-08-01 16:23:56 +0000   
1     2017-08-01 00:17:27 +0000   
2     2017-07-31 00:18:03 +0000   
3     2017-07-30 15:58:51 +0000   
4     2017-07-29 16:00:24 +0000   
...                         ...   
2351  2015-11-16 00:24:50 +0000   
2352  2015-11-16 00:04:52 +0000   
2353  2015-11-15 23:21:54 +0000   
2354  2015-11-15 23:05:30 +0000   
2355  2015-11-15 22:32:08 +0000   

                                                 source  \
0     <a href="http://twitter.com/download/iphone" r...   
1     <a href="http://twitter.com/download/iphone" r...   
2     <a href="http://twitter.com/download/iphone" r...   
3     <a href="http://twitter.com/download/iphone" r...   
4     <a href="http://twitter.com/download/iphone" r...   
...                                                 ...   
2351  <a href="http://twitter.com/download/iphone" r...   
2352  <a href="http://twitter.com/download/iphone" r...   
2353  <a href="http://twitter.com/download/iphone" r...   
2354  <a href="http://twitter.com/download/iphone" r...   
2355  <a href="http://twitter.com/download/iphone" r...   

                                                   text  retweeted_status_id  \
0     This is Phineas. He's a mystical boy. Only eve...                  NaN   
1     This is Tilly. She's just checking pup on you....                  NaN   
2     This is Archie. He is a rare Norwegian Pouncin...                  NaN   
3     This is Darla. She commenced a snooze mid meal...                  NaN   
4     This is Franklin. He would like you to stop ca...                  NaN   
...                                                 ...                  ...   
2351  Here we have a 1949 1st generation vulpix. Enj...                  NaN   
2352  This is a purebred Piers Morgan. Loves to Netf...                  NaN   
2353  Here is a very happy pup. Big fan of well-main...                  NaN   
2354  This is a western brown Mitsubishi terrier. Up...                  NaN   
2355  Here we have a Japanese Irish Setter. Lost eye...                  NaN   

      retweeted_status_user_id retweeted_status_timestamp  \
0                          NaN                        NaN   
1                          NaN                        NaN   
2                          NaN                        NaN   
3                          NaN                        NaN   
4                          NaN                        NaN   
...                        ...                        ...   
2351                       NaN                        NaN   
2352                       NaN                        NaN   
2353                       NaN                        NaN   
2354                       NaN                        NaN   
2355                       NaN                        NaN   

                                          expanded_urls  rating_numerator  \
0     https://twitter.com/dog_rates/status/892420643...                13   
1     https://twitter.com/dog_rates/status/892177421...                13   
2     https://twitter.com/dog_rates/status/891815181...                12   
3     https://twitt

In [10]:
#there are not duplicated tweets_id
len(pd.unique(df_dogs['tweet_id']))

2356

In [11]:
# there are a lot of names that were not found that could be fixed
df_dogs['name'].value_counts()

None        745
a            55
Charlie      12
Oliver       11
Cooper       11
           ... 
Bluebert      1
Divine        1
Dobby         1
Hall          1
Mason         1
Name: name, Length: 957, dtype: int64

In [12]:
# seems like there are issues with the numerators, understanding that  people tend to rate over 10 

df_dogs['rating_numerator'].value_counts()

12      558
11      464
10      461
13      351
9       158
8       102
7        55
14       54
5        37
6        32
3        19
4        17
1         9
2         9
420       2
0         2
15        2
75        2
80        1
20        1
24        1
26        1
44        1
50        1
60        1
165       1
84        1
88        1
144       1
182       1
143       1
666       1
960       1
1776      1
17        1
27        1
45        1
99        1
121       1
204       1
Name: rating_numerator, dtype: int64

In [13]:
# seems like there are issues with the denominators, understanding that  denominator is usually 10.

df_dogs['rating_denominator'].value_counts()

10     2333
11        3
50        3
80        2
20        2
2         1
16        1
40        1
70        1
15        1
90        1
110       1
120       1
130       1
150       1
170       1
7         1
0         1
Name: rating_denominator, dtype: int64

In [14]:
pictures_df

tweet_id                                          jpg_url  \
0     666020888022790149  https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg   
1     666029285002620928  https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg   
2     666033412701032449  https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg   
3     666044226329800704  https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg   
4     666049248165822465  https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg   
...                  ...                                              ...   
2070  891327558926688256  https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg   
2071  891689557279858688  https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg   
2072  891815181378084864  https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg   
2073  892177421306343426  https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg   
2074  892420643555336193  https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg   

      img_num                      p1   p1_conf  p1_dog                  p2  \
0           1  Welsh_springer_spaniel  0.465074    True              collie   
1           1                 redbone  0.506826    True  miniature_pinscher   
2           1         German_shepherd  0.596461    True            malinois   
3           1     Rhodesian_ridgeback  0.408143    True             redbone   
4           1      miniature_pinscher  0.560311    True          Rottweiler   
...       ...                     ...       ...     ...                 ...   
2070        2                  basset  0.555712    True    English_springer   
2071        1             paper_towel  0.170278   False  Labrador_retriever   
2072        1               Chihuahua  0.716012    True            malamute   
2073        1               Chihuahua  0.323581    True            Pekinese   
2074        1                  orange  0.097049   False               bagel   

       p2_conf  p2_dog                           p3   p3_conf  p3_dog  
0     0.156665    True            Shetland_sheepdog  0.061428    True  
1     0.074192    True          Rhodesian_ridgeback  0.072010    True  
2     0.138584    True                   bloodhound  0.116197    True  
3     0.360687    True           miniature_pinscher  0.222752    True  
4     0.243682    True                     Doberman  0.154629    True  
...        ...     ...                          ...       ...     ...  
2070  0.225770    True  German_short-haired_pointer  0.175219    True  
2071  0.168086    True                      spatula  0.040836   False  
2072  0.078253    True                       kelpie  0.031379    True  
2073  0.090647    True                     papillon  0.068957    True  
2074  0.085851   False                       banana  0.076110   False  

[2075 rows x 12 columns]

In [15]:
#there are not duplicated tweets_id
len(pd.unique(pictures_df['tweet_id']))

2075

In [16]:
# there are pictures that the algorithm recognize not as a dog
pictures_df['p1'].value_counts()

golden_retriever      150
Labrador_retriever    100
Pembroke               89
Chihuahua              83
pug                    57
                     ... 
microphone              1
slug                    1
snowmobile              1
water_bottle            1
hare                    1
Name: p1, Length: 378, dtype: int64

In [17]:
tweets_df

id  retweet_count  favorite_count
0     892420643555336193           7372           35059
1     892177421306343426           5493           30355
2     891815181378084864           3631           22814
3     891689557279858688           7549           38328
4     891327558926688256           8136           36602
...                  ...            ...             ...
2326  666049248165822465             38              94
2327  666044226329800704            122             259
2328  666033412701032449             39             107
2329  666029285002620928             41             118
2330  666020888022790149            444            2344

[2331 rows x 3 columns]

In [18]:
len(pd.unique(tweets_df['id']))

2331

In [19]:
df_dogs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

In [20]:
tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2331 entries, 0 to 2330
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   id              2331 non-null   int64
 1   retweet_count   2331 non-null   int64
 2   favorite_count  2331 non-null   int64
dtypes: int64(3)
memory usage: 54.8 KB


In [21]:
pictures_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


### Cleaning

In [22]:
dogs_clean = df_dogs.copy()
tweets_clean = tweets_df.copy()
pictures_clean = pictures_df.copy()

#### Define issue (1)

* separing time zone from time in the timestamp column from dogs_clean

#### Clean issue (1)

In [23]:
dogs_clean['timestamp']=dogs_clean['timestamp'].str.split('\+0000', expand=True)[0]

#### Test cleaned issue (1)

In [24]:
dogs_clean['timestamp']

0       2017-08-01 16:23:56 
1       2017-08-01 00:17:27 
2       2017-07-31 00:18:03 
3       2017-07-30 15:58:51 
4       2017-07-29 16:00:24 
                ...         
2351    2015-11-16 00:24:50 
2352    2015-11-16 00:04:52 
2353    2015-11-15 23:21:54 
2354    2015-11-15 23:05:30 
2355    2015-11-15 22:32:08 
Name: timestamp, Length: 2356, dtype: object

#### Define issue (2)

* converting the timestamp into date time type

#### Clean issue (2)

In [25]:
dogs_clean['timestamp'] =  pd.to_datetime(dogs_clean['timestamp'])

#### Test cleaned issue (2)

In [26]:
dogs_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   tweet_id                    2356 non-null   int64         
 1   in_reply_to_status_id       78 non-null     float64       
 2   in_reply_to_user_id         78 non-null     float64       
 3   timestamp                   2356 non-null   datetime64[ns]
 4   source                      2356 non-null   object        
 5   text                        2356 non-null   object        
 6   retweeted_status_id         181 non-null    float64       
 7   retweeted_status_user_id    181 non-null    float64       
 8   retweeted_status_timestamp  181 non-null    object        
 9   expanded_urls               2297 non-null   object        
 10  rating_numerator            2356 non-null   int64         
 11  rating_denominator          2356 non-null   int64       

#### Define issue (3)

* column doggo , floofer, pupper & puppo : they could be stored in one column (categorical one), so we need to concatenate all the columns into one

#### Clean issue (3)

In [27]:
dogs_clean['doggo'].replace('None' , np.nan , inplace=True)
dogs_clean['floofer'].replace('None' , np.nan , inplace=True)
dogs_clean['pupper'].replace('None' , np.nan , inplace=True)
dogs_clean['puppo'].replace('None' , np.nan , inplace=True)

dogs_clean['dog_stage'] = (dogs_clean['doggo'].map(str) + ',' +dogs_clean['floofer'].map(str)
                            + ',' +dogs_clean['pupper'].map(str) + ',' +dogs_clean['puppo'].map(str))
dogs_clean['dog_stage'] = dogs_clean['dog_stage'].str.replace('nan,','').str.replace(',nan','').str.replace('nan','')

indexes = dogs_clean.index[dogs_clean['dog_stage'] == ''].tolist()

dogs_clean = dogs_clean.drop(indexes)

dogs_clean = dogs_clean.reset_index(drop = True)


#### Test cleaned issue (3)

In [28]:
dogs_clean['dog_stage'].value_counts()

pupper           245
doggo             83
puppo             29
doggo,pupper      12
floofer            9
doggo,puppo        1
doggo,floofer      1
Name: dog_stage, dtype: int64

#### Define issue (4)

* column rating_numerator: there are values that donnot correspond to what the text says so we will extract the data again for numerator and denominator and then converting it into float type

#### Clean issue (4)

In [29]:
dogs_clean['test'] = dogs_clean.text.str.extract(r'(\d+\.\d*\/\d+|\d+\/\d+)')

dogs_clean[['rating_numerator','rating_denominator']]= dogs_clean['test'].str.split('/', expand = True)

dogs_clean = dogs_clean.drop(['test'], axis = 1)

dogs_clean['rating_numerator'] = dogs_clean['rating_numerator'].astype(float)
dogs_clean['rating_denominator'] = dogs_clean['rating_denominator'].astype(float)

dogs_clean.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   tweet_id                    380 non-null    int64         
 1   in_reply_to_status_id       8 non-null      float64       
 2   in_reply_to_user_id         8 non-null      float64       
 3   timestamp                   380 non-null    datetime64[ns]
 4   source                      380 non-null    object        
 5   text                        380 non-null    object        
 6   retweeted_status_id         36 non-null     float64       
 7   retweeted_status_user_id    36 non-null     float64       
 8   retweeted_status_timestamp  36 non-null     object        
 9   expanded_urls               374 non-null    object        
 10  rating_numerator            380 non-null    float64       
 11  rating_denominator          380 non-null    float64       

#### Test cleaned issue (4)

In [30]:
dogs_clean['rating_numerator'].value_counts()

12.00    97
11.00    84
10.00    67
13.00    64
14.00    19
8.00     18
9.00     17
7.00      9
5.00      2
11.27     1
6.00      1
3.00      1
Name: rating_numerator, dtype: int64

#### Define issue (5)

* column name in df_dogs : there are names that are not names, all of them start with lower case so we will replace them with 'None'

#### Clean issue (5)

In [31]:
i=0
names_replace= []
for name in dogs_clean['name']:
    
    if name[0].islower():
        names_replace.append(name)
    else:
        pass
    i+=1

In [32]:
dogs_clean = dogs_clean.replace(names_replace , 'None')

#### Test cleaned issue (5)

In [33]:
dogs_clean['name'].value_counts()

None         176
Sampson        4
Bo             4
Cooper         3
Oliver         3
            ... 
Olivia         1
Lennon         1
Otis           1
Grizzwald      1
Aqua           1
Name: name, Length: 174, dtype: int64

#### Define issue (6)

* retweets : we should drop the retweets and replays

#### Clean issue (6)

In [34]:
# dropping tweets that are retweets in dogs_clean 
dogs_clean = dogs_clean[dogs_clean.retweeted_status_id.isnull()]
dogs_clean = dogs_clean[dogs_clean.in_reply_to_status_id.isnull()]

#### Test cleaned issue (6)

In [35]:
dogs_clean['retweeted_status_id'].value_counts()

Series([], Name: retweeted_status_id, dtype: int64)

#### Define issue (7) : 

* pictures_df : we have rows where all the pictures do not contain dogs, so we will drop those rows

#### Clean issue (7)

In [36]:
index = pictures_clean[(pictures_clean['p1_dog'] == False) & (pictures_clean['p2_dog'] == False) & (pictures_clean['p3_dog'] == False)].index
pictures_clean = pictures_clean.drop(index)

#### Test cleaned issue (7)

In [37]:
pictures_clean['p1_dog'].value_counts()

True     1532
False     219
Name: p1_dog, dtype: int64

In [38]:
pictures_clean['p2_dog'].value_counts()

True     1553
False     198
Name: p2_dog, dtype: int64

In [39]:
pictures_clean['p3_dog'].value_counts()

True     1499
False     252
Name: p3_dog, dtype: int64

#### Define issue (8)

* breed in pictures_df : we will choose the most accurate breed according to the algorithm iterations

#### Clean issue (8)

In [40]:
# selecting the most accurate breed of dog. 

conditions = ([(pictures_clean['p1_dog'] == True) &(pictures_clean['p1_conf'] >= pictures_clean['p2_conf']) & (pictures_clean['p1_conf'] >= pictures_clean['p3_conf']),
             (pictures_clean['p2_dog'] == True) & (pictures_clean['p2_conf'] >= pictures_clean['p1_conf']) & (pictures_clean['p2_conf'] >= pictures_clean['p3_conf']),
              (pictures_clean['p3_dog'] == True) & (pictures_clean['p3_conf'] >= pictures_clean['p1_conf']) & (pictures_clean['p3_conf'] >= pictures_clean['p2_conf'])])
choice = [pictures_clean['p1'] , pictures_clean['p2'], pictures_clean['p3']]

pictures_clean['breed'] = np.select(conditions , choice , default = np.nan)

#### Test cleaned issue (8)

In [41]:
pictures_clean['breed']

0         Welsh_springer_spaniel
1                        redbone
2                German_shepherd
3            Rhodesian_ridgeback
4             miniature_pinscher
                  ...           
2069    Chesapeake_Bay_retriever
2070                      basset
2071                         NaN
2072                   Chihuahua
2073                   Chihuahua
Name: breed, Length: 1751, dtype: object

#### Define issue (9)

#### Clean issue (9)

#### Test cleaned issue (9)

#### Define issue (10)

* mergin the DFs into the master DF so we drop the tweet ids that don't match in all the DFs, but before we drop the columns that we are not going to use

#### Clean issue (10)

In [42]:
#dropping 'useless' columns in pictures_clean

pictures_clean = pictures_clean.drop(['p1','p2','p3','p1_conf','p2_conf','p3_conf', 'p1_dog','p2_dog','p3_dog' , 'img_num'] , axis = 1)
pictures_clean = pictures_clean.dropna()

In [43]:
# dropping the 'useless' columns in dogs_clean
dogs_clean = dogs_clean.drop(['retweeted_status_id','retweeted_status_user_id','retweeted_status_timestamp' ,
                              'in_reply_to_status_id', 'in_reply_to_user_id','expanded_urls','source','doggo','floofer','puppo','pupper'], axis = 1)

In [44]:
tweets_clean = tweets_clean.rename(columns={'id' : 'tweet_id'})
dfs = [dogs_clean , tweets_clean , pictures_clean]
master_df = reduce(lambda left,right: pd.merge(left,right,on='tweet_id'), dfs)

#### Test cleaned issue (10)

In [45]:
master_df

tweet_id           timestamp  \
0    890240255349198849 2017-07-26 15:59:51   
1    889665388333682689 2017-07-25 01:55:32   
2    889531135344209921 2017-07-24 17:02:04   
3    886366144734445568 2017-07-15 23:25:31   
4    884162670584377345 2017-07-09 21:29:42   
..                  ...                 ...   
221  672622327801233409 2015-12-04 03:43:54   
222  672594978741354496 2015-12-04 01:55:13   
223  672481316919734272 2015-12-03 18:23:34   
224  672254177670729728 2015-12-03 03:21:00   
225  672160042234327040 2015-12-02 21:06:56   

                                                  text  rating_numerator  \
0    This is Cassie. She is a college pup. Studying...              14.0   
1    Here's a puppo that seems to be on the fence a...              13.0   
2    This is Stuart. He's sporting his favorite fan...              13.0   
3    This is Roscoe. Another pupper fallen victim t...              12.0   
4    Meet Yogi. He doesn't have any important dog m...              12.0   
..                                                 ...               ...   
221  This lil pupper is sad because we haven't foun...              12.0   
222  Meet Scott. Just trying to catch his train to ...               9.0   
223  Say hello to Jazz. She should be on the cover ...              12.0   
224  This is Rolf. He's having the time of his life...              11.0   
225  This is Bubba. He's a Titted Peebles Aorta. Ev...               8.0   

     rating_denominator    name dog_stage  retweet_count  favorite_count  \
0                  10.0  Cassie     doggo           6397           28984   
1                  10.0    None     puppo           8754           43639   
2                  10.0  Stuart     puppo           1971           13830   
3                  10.0  Roscoe    pupper           2767           19215   
4                  10.0    Yogi     doggo           2619           18578   
..                  ...     ...       ...            ...             ...   
221                10.0    None    pupper            442            1201   
222                10.0   Scott    pupper            564            1239   
223                10.0    Jazz    pupper            120             672   
224                10.0    Rolf    pupper            682            1332   
225                10.0   Bubba    pupper            328             819   

                                             jpg_url             breed  
0    https://pbs.twimg.com/media/DFrEyVuW0AAO3t9.jpg          Pembroke  
1    https://pbs.twimg.com/media/DFi579UWsAAatzw.jpg          Pembroke  
2    https://pbs.twimg.com/media/DFg_2PVW0AEHN3p.jpg  golden_retriever  
3    https://pbs.twimg.com/media/DE0BTnQUwAApKEH.jpg    French_bulldog  
4    https://pbs.twimg.com/media/DEUtQbzW0AUTv_o.jpg   German_shepherd  
..                                               ...               ...  
221  https://pbs.twimg.com/media/CVWicBbUYAIomjC.jpg  golden_retriever  
222  https://pbs.twimg.com/media/CVWJkJXWsAInlZl.jpg    Great_Pyrenees  
223  https://pbs.twimg.com/media/CVUiMUeW4AEQgkU.jpg     Border_collie  
224  https://pbs.twimg.com/media/CVRTmz1WcAA4uMF.jpg               pug  
225  https://pbs.twimg.com/media/CVP9_beUEAAwURR.jpg               pug  

[226 rows x 11 columns]

## Storing, Analyzing and Visualizing

### Storing

In [46]:
# storing the clean data frame

master_df.to_csv('twitter_archive_master.csv', index=False)

In [47]:
master_df.describe()

tweet_id  rating_numerator  rating_denominator  retweet_count  \
count  2.260000e+02        226.000000               226.0     226.000000   
mean   7.517461e+17         11.306504                10.0    3610.955752   
std    6.815178e+16          1.499394                 0.0    7797.702703   
min    6.721600e+17          5.000000                10.0      81.000000   
25%    6.866345e+17         10.000000                10.0     819.250000   
50%    7.388878e+17         11.000000                10.0    1517.000000   
75%    8.147175e+17         12.000000                10.0    3069.500000   
max    8.902403e+17         14.000000                10.0   74394.000000   

       favorite_count  
count      226.000000  
mean     11549.601770  
std      19198.997674  
min        607.000000  
25%       2534.500000  
50%       5603.000000  
75%      11751.250000  
max     150726.000000

In [48]:
master_df['text'][master_df['rating_numerator'].idxmin()]

"Please stop sending it pictures that don't even have a doggo or pupper in them. Churlish af. 5/10 neat couch tho https://t.co/u2c9c7qSg8"

In [49]:
master_df['text'][master_df['rating_numerator'].idxmax()]

'This is Cassie. She is a college pup. Studying international doggo communication and stick theory. 14/10 so elegant much sophisticate https://t.co/t1bfwz5S2A'

### Analyzing & Visualizing

In [50]:
#Understanding most-liked dog's breed
master_df.loc[master_df['favorite_count']<20000 , 'breed'] = 'other breeds' #representing just the most relevant breeds
fig_a = px.pie(master_df, values = 'favorite_count' , names = 'breed')
fig_a.update_layout(title = {'text' : "Dog's breed by favorite count", 'y' : 0.95, 'x': 0.5 , 'xanchor':'center' , 'yanchor' : 'top'})
fig_a.show()

### From the graph before we can see that "big" dog's breed are the most liked ones from this data set, with a small difference between them, Labrador and Golden are also quite similar physically speaking

In [51]:
#Understanding most-retweeted dog's breed
master_df.loc[master_df['retweet_count']<5000 , 'breed'] = 'other breeds' #representing just the most relevant breeds
fig_b = px.pie(master_df, values = 'retweet_count' , names = 'breed')
fig_b.update_layout(title = {'text' : "Dog's breed by retweet count", 'y' : 0.95, 'x': 0.5 , 'xanchor':'center' , 'yanchor' : 'top'})
fig_b.show()

### Labrador retriever is by far the most retweet dog which follows the tendency of being also the most-liked dog

In [52]:
#Understanding most-retweeted dog's stage
fig_c = px.pie(master_df, values = 'retweet_count' , names = 'dog_stage')
fig_c.update_layout(title = {'text' : "Dog's stage by retweet count", 'y' : 0.95, 'x': 0.5 , 'xanchor':'center' , 'yanchor' : 'top'})
fig_c.show()

### Doggo and pupper are the most retweeted stages, while intermediate stages are not that retweeted

In [53]:
#Understanding most-liked dog's stage
fig_d = px.pie(master_df, values = 'favorite_count' , names = 'dog_stage')
fig_d.update_layout(title = {'text' : "Dog's stage by favorite count", 'y' : 0.95, 'x': 0.5 , 'xanchor':'center' , 'yanchor' : 'top'})
fig_d.show()

### Pupper is the most-liked stage from this data set, floofer is the less-liked stage